In [ ]:
from sklearn.model_selection import train_test_split
import json

In [ ]:
import sys
from google.colab import drive
drive.mount('/content/drive')
sys.path.append('/content/drive/MyDrive/lattcie ner/fyz-lattcie/fyz_lattice_NER')

sys.argv=['']
del sys

Mounted at /content/drive


In [ ]:
with open('/content/drive/MyDrive/cyx_数据/问句汇总.json',encoding="utf-8") as file_obj:
  ques_all=json.load(file_obj)

In [ ]:
ques_clean=[]
for i in ques_all:
  ques_clean.append([list(i),ques_all[i]])

In [ ]:
train_set,test_set=train_test_split(ques_clean,test_size=0.4)
dev_set,test_set=train_test_split(test_set,test_size=0.4)

In [ ]:
f=open('/content/drive/MyDrive/lattcie ner/fyz-lattcie/cyx/final/train.txt','w',encoding='utf-8')
for i in train_set:
    for index in range(0,len(i[0])):
        f.write(i[0][index]+'	'+i[1][index]+'\n')
    f.write('\n')
f.close()

f=open('/content/drive/MyDrive/lattcie ner/fyz-lattcie/cyx/final/dev.txt','w',encoding='utf-8')
for i in dev_set:
    for index in range(0,len(i[0])):
        f.write(i[0][index]+'	'+i[1][index]+'\n')
    f.write('\n')
f.close()

f=open('/content/drive/MyDrive/lattcie ner/fyz-lattcie/cyx/final/test.txt','w',encoding='utf-8')
for i in test_set:
    for index in range(0,len(i[0])):
        f.write(i[0][index]+'	'+i[1][index]+'\n')
    f.write('\n')
f.close()

In [ ]:
import time
import sys
import argparse
import random
import copy
import torch
import json
import gc
import pickle
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from utils.metric import get_ner_fmeasure
from model.bilstmcrf import BiLSTM_CRF as SeqModel
from utils.data import Data

seed_num = 100
random.seed(seed_num)
torch.manual_seed(seed_num)
np.random.seed(seed_num)


def data_initialization(data, gaz_file, train_file, dev_file, test_file):
    data.build_alphabet(train_file)
    data.build_alphabet(dev_file)
    data.build_alphabet(test_file)
    data.build_gaz_file(gaz_file)

    #gaz_alphabet train,dev,test file在embedding中匹配到的词语
    data.build_gaz_alphabet(train_file)
    data.build_gaz_alphabet(dev_file)
    data.build_gaz_alphabet(test_file)
    data.fix_alphabet()
    return data


def predict_check(pred_variable, gold_variable, mask_variable):
    """
        input:
            pred_variable (batch_size, sent_len): pred tag result, in numpy format
            gold_variable (batch_size, sent_len): gold  result variable
            mask_variable (batch_size, sent_len): mask variable
    """
    pred = pred_variable.cpu().data.numpy()
    gold = gold_variable.cpu().data.numpy()
    mask = mask_variable.cpu().data.numpy()
    overlaped = (pred == gold)
    right_token = np.sum(overlaped * mask)
    total_token = mask.sum()
    # print("right: %s, total: %s"%(right_token, total_token))
    return right_token, total_token


def recover_label(pred_variable, gold_variable, mask_variable, label_alphabet, word_recover):
    """
        input:
            pred_variable (batch_size, sent_len): pred tag result
            gold_variable (batch_size, sent_len): gold result variable
            mask_variable (batch_size, sent_len): mask variable
    """
    
    pred_variable = pred_variable[word_recover]
    gold_variable = gold_variable[word_recover]
    mask_variable = mask_variable[word_recover]
    batch_size = gold_variable.size(0)
    seq_len = gold_variable.size(1)
    mask = mask_variable.cpu().data.numpy()
    pred_tag = pred_variable.cpu().data.numpy()
    gold_tag = gold_variable.cpu().data.numpy()
    batch_size = mask.shape[0]
    pred_label = []
    gold_label = []
    for idx in range(batch_size):
        pred = [label_alphabet.get_instance(pred_tag[idx][idy]) for idy in range(seq_len) if mask[idx][idy] != 0]
        gold = [label_alphabet.get_instance(gold_tag[idx][idy]) for idy in range(seq_len) if mask[idx][idy] != 0]
        # print "p:",pred, pred_tag.tolist()
        # print "g:", gold, gold_tag.tolist()
        assert(len(pred)==len(gold))
        pred_label.append(pred)
        gold_label.append(gold)
    return pred_label, gold_label


def save_data_setting(data, save_file):
    """
    new_data = copy.deepcopy(data)
    ## remove input instances
    new_data.train_texts = []
    new_data.dev_texts = []
    new_data.test_texts = []
    new_data.raw_texts = []
    new_data.train_Ids = []
    new_data.dev_Ids = []
    new_data.test_Ids = []
    new_data.raw_Ids = []"""
    ## save data settings
    with open(save_file, 'wb') as fp:
        pickle.dump(data, fp)
    print ("Data setting saved to file: ", save_file)


def load_data_setting(save_file):
    with open(save_file, 'rb') as fp:
        data = pickle.load(fp)
    print ("Data setting loaded from file: ", save_file)
    data.show_data_summary()
    return data

def lr_decay(optimizer, epoch, decay_rate, init_lr):
    lr = init_lr * ((1-decay_rate)**epoch)
    print (" Learning rate is setted as:", lr)
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    return optimizer



def evaluate(data, model, name):
    if name == "train":
        instances = data.train_Ids
    elif name == "dev":
        instances = data.dev_Ids
    elif name == 'test':
        instances = data.test_Ids
    elif name == 'raw':
        instances = data.raw_Ids
    else:
        print ("Error: wrong evaluate name,", name)
    pred_results = []
    gold_results = []
    ## set model in eval model
    model.eval()
    batch_size = 10
    start_time = time.time()
    train_num = len(instances)
    total_batch = train_num//batch_size+1
    for batch_id in range(total_batch):
        start = batch_id*batch_size
        end = (batch_id+1)*batch_size 
        if end >train_num:
            end =  train_num
        instance = instances[start:end]
        if not instance:
            continue
        gaz_list,batch_word, batch_biword, batch_wordlen, batch_wordrecover, batch_char, batch_charlen, batch_charrecover, batch_label, mask  = batchify_with_label(instance, data.HP_gpu, True)
        tag_seq = model(gaz_list,batch_word, batch_biword, batch_wordlen, batch_char, batch_charlen, batch_charrecover, mask)
        # print "tag:",tag_seq
        pred_label, gold_label = recover_label(tag_seq, batch_label, mask, data.label_alphabet, batch_wordrecover)
        pred_results += pred_label
        gold_results += gold_label
    decode_time = time.time() - start_time
    speed = len(instances)/decode_time
    acc, p, r, f, right_num = get_ner_fmeasure(gold_results, pred_results, data.tagScheme)
    return speed, acc, p, r, f, pred_results, right_num


def batchify_with_label(input_batch_list, gpu, volatile_flag=False):
    """
        input: list of words, chars and labels, various length. [[words,biwords,chars,gaz, labels],[words,biwords,chars,labels],...]
            words: word ids for one sentence. (batch_size, sent_len) 
            chars: char ids for on sentences, various length. (batch_size, sent_len, each_word_length)
        output:
            zero padding for word and char, with their batch length
            word_seq_tensor: (batch_size, max_sent_len) Variable
            word_seq_lengths: (batch_size,1) Tensor
            char_seq_tensor: (batch_size*max_sent_len, max_word_len) Variable
            char_seq_lengths: (batch_size*max_sent_len,1) Tensor
            char_seq_recover: (batch_size*max_sent_len,1)  recover char sequence order 
            label_seq_tensor: (batch_size, max_sent_len)
            mask: (batch_size, max_sent_len) 
    """
    batch_size = len(input_batch_list)
    words = [sent[0] for sent in input_batch_list]
    biwords = [sent[1] for sent in input_batch_list]
    chars = [sent[2] for sent in input_batch_list]


    gazs = [sent[3] for sent in input_batch_list]
    labels = [sent[4] for sent in input_batch_list]
    word_seq_lengths = torch.LongTensor(list(map(len, words)))
    max_seq_len = word_seq_lengths.max().item()

    word_seq_tensor = autograd.Variable(torch.zeros((batch_size, max_seq_len)), volatile =  volatile_flag).long()
    biword_seq_tensor = autograd.Variable(torch.zeros((batch_size, max_seq_len)), volatile =  volatile_flag).long()
    label_seq_tensor = autograd.Variable(torch.zeros((batch_size, max_seq_len)),volatile =  volatile_flag).long()
    mask = autograd.Variable(torch.zeros((batch_size, max_seq_len)),volatile =  volatile_flag).byte()

    for idx, (seq, biseq, label, seqlen) in enumerate(zip(words, biwords, labels, word_seq_lengths)):
        word_seq_tensor[idx, :seqlen] = torch.LongTensor(seq)
        biword_seq_tensor[idx, :seqlen] = torch.LongTensor(biseq)
        label_seq_tensor[idx, :seqlen] = torch.LongTensor(label)
        mask[idx, :seqlen] = torch.Tensor([1]*seqlen.item())

    word_seq_lengths, word_perm_idx = word_seq_lengths.sort(0, descending=True)
    word_seq_tensor = word_seq_tensor[word_perm_idx]
    biword_seq_tensor = biword_seq_tensor[word_perm_idx]
    label_seq_tensor = label_seq_tensor[word_perm_idx]
    mask = mask[word_perm_idx]

    ### deal with char
    # pad_chars (batch_size, max_seq_len)
    pad_chars = [chars[idx] + [[0]] * (max_seq_len-len(chars[idx])) for idx in range(len(chars))]
    length_list = [list(map(len, pad_char)) for pad_char in pad_chars]
    #length_list = [len(pad_char) for pad_char in pad_chars]
    max_word_len = max(map(max, length_list))
    char_seq_tensor = autograd.Variable(torch.zeros((batch_size, max_seq_len, max_word_len)), volatile =  volatile_flag).long()
    char_seq_lengths = torch.LongTensor(length_list)
    for idx, (seq, seqlen) in enumerate(zip(pad_chars, char_seq_lengths)):
        for idy, (word, wordlen) in enumerate(zip(seq, seqlen)):
            # print len(word), wordlen
            char_seq_tensor[idx, idy, :wordlen] = torch.LongTensor(word)
    char_seq_tensor = char_seq_tensor[word_perm_idx].view(batch_size*max_seq_len,-1)
    char_seq_lengths = char_seq_lengths[word_perm_idx].view(batch_size*max_seq_len,)
    char_seq_lengths, char_perm_idx = char_seq_lengths.sort(0, descending=True)
    char_seq_tensor = char_seq_tensor[char_perm_idx]
    _, char_seq_recover = char_perm_idx.sort(0, descending=False)
    _, word_seq_recover = word_perm_idx.sort(0, descending=False)
    
    ## keep the gaz_list in orignial order
    
    gaz_list = [ gazs[i] for i in word_perm_idx]
    gaz_list.append(volatile_flag)
    if gpu:
        word_seq_tensor = word_seq_tensor.cuda()
        biword_seq_tensor = biword_seq_tensor.cuda()
        word_seq_lengths = word_seq_lengths.cuda()
        word_seq_recover = word_seq_recover.cuda()
        label_seq_tensor = label_seq_tensor.cuda()
        char_seq_tensor = char_seq_tensor.cuda()
        char_seq_recover = char_seq_recover.cuda()
        mask = mask.cuda()
    return gaz_list, word_seq_tensor, biword_seq_tensor, word_seq_lengths, word_seq_recover, char_seq_tensor, char_seq_lengths, char_seq_recover, label_seq_tensor, mask


def train(data, save_model_dir,save_data_set, seg=True):
    dev_log=[]
    #test_log=[]
    train_log=[]
    print ("Training model...")
    data.show_data_summary()
    #save_data_name = save_data_set
    #save_data_setting(data, save_data_name)

    save_data_name = save_data_set
    save_data_setting(data, save_data_name)

    model = SeqModel(data)
    print ("finished built model.")
    loss_function = nn.NLLLoss()
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = optim.SGD(parameters, lr=data.HP_lr, momentum=data.HP_momentum)
    best_dev = -1
    #data.HP_iteration = 100#epoch次数
    data.HP_iteration = 25#epoch次数
    ## start training data.HP_iteration
    for idx in range(data.HP_iteration):
        epoch_start = time.time()
        temp_start = epoch_start
        print("Epoch: %s/%s" %(idx,data.HP_iteration))
        optimizer = lr_decay(optimizer, idx, data.HP_lr_decay, data.HP_lr)
        instance_count = 0
        sample_id = 0
        sample_loss = 0
        batch_loss = 0
        total_loss = 0
        right_token = 0
        whole_token = 0
        random.shuffle(data.train_Ids)
        ## set model in train model
        model.train()
        model.zero_grad()
        batch_size = 10 ## current only support batch size = 1 to compulate and accumulate to data.HP_batch_size update weights
        train_num = len(data.train_Ids)
        total_batch = train_num//batch_size+1
        for batch_id in range(total_batch):
            start = batch_id*batch_size
            end = (batch_id+1)*batch_size 
            if end >train_num:
                end = train_num
            instance = data.train_Ids[start:end]
            if not instance:
                continue
            gaz_list,  batch_word, batch_biword, batch_wordlen, batch_wordrecover, batch_char, batch_charlen, batch_charrecover, batch_label, mask  = batchify_with_label(instance, data.HP_gpu)
            # print "gaz_list:",gaz_list
            # exit(0)
            instance_count += 1
            loss, tag_seq = model.neg_log_likelihood_loss(gaz_list, batch_word, batch_biword, batch_wordlen, batch_char, batch_charlen, batch_charrecover, batch_label, mask)
            right, whole = predict_check(tag_seq, batch_label, mask)
            right_token += right
            whole_token += whole
            #sample_loss += loss.data[0]
            #total_loss += loss.data[0]
            sample_loss += loss.data.item()
            total_loss += loss.data.item()
            batch_loss += loss

            #if end%2000 == 0:
            if end%500 == 0:
                temp_time = time.time()
                temp_cost = temp_time - temp_start
                temp_start = temp_time
                print("Instance: %s; Time: %.2fs; loss: %.4f; acc: %s/%s=%.4f"%(end, temp_cost, sample_loss, right_token, whole_token,(right_token+0.)/whole_token))
                sys.stdout.flush()
                sample_loss = 0
            if end%data.HP_batch_size == 0:
                batch_loss.backward()
                optimizer.step()
                model.zero_grad()
                batch_loss = 0
        train_log.append(right_token)
        temp_time = time.time()
        temp_cost = temp_time - temp_start
        print("     Instance: %s; Time: %.2fs; loss: %.4f; acc: %s/%s=%.4f"%(end, temp_cost, sample_loss, right_token, whole_token,(right_token+0.)/whole_token))       
        epoch_finish = time.time()
        epoch_cost = epoch_finish - epoch_start
        print("Epoch: %s training finished. Time: %.2fs, speed: %.2fst/s,  total loss: %s"%(idx, epoch_cost, train_num/epoch_cost, total_loss))
        
        # exit(0)
        # continue
        speed, acc, p, r, f, _, right_num = evaluate(data, model, "dev")
        dev_log.append(right_num)
        dev_finish = time.time()
        dev_cost = dev_finish - epoch_finish

        if seg:
            current_score = f
            print("Dev: time: %.2fs, speed: %.2fst/s; acc: %.4f, p: %.4f, r: %.4f, f: %.4f"%(dev_cost, speed, acc, p, r, f))
        else:
            current_score = acc
            print("Dev: time: %.2fs speed: %.2fst/s; acc: %.4f"%(dev_cost, speed, acc))
        
        if current_score > best_dev:
            if seg:
                print ("Exceed previous best f score:", best_dev)
            else:
                print ("Exceed previous best acc score:", best_dev)

            model_name = save_model_dir
            torch.save(model.state_dict(), model_name)
            #model_name = save_model_dir +'.'+ str(idx) + ".model"
            #torch.save(model.state_dict(), model_name)
            best_dev = current_score 
        # ## decode test
        speed, acc, p, r, f, _, right_num = evaluate(data, model, "test")
        #test_log.append(_)
        test_finish = time.time()
        test_cost = test_finish - dev_finish
        if seg:
            print("Test: time: %.2fs, speed: %.2fst/s; acc: %.4f, p: %.4f, r: %.4f, f: %.4f"%(test_cost, speed, acc, p, r, f))
        else:
            print("Test: time: %.2fs, speed: %.2fst/s; acc: %.4f"%(test_cost, speed, acc))
        
        gc.collect()
    return model,train_log,dev_log


def load_model_decode(model_dir, data, name, gpu, seg=True):
    data.HP_gpu = gpu
    print ("Load Model from file: ", model_dir)
    model = SeqModel(data)
    ## load model need consider if the model trained in GPU and load in CPU, or vice versa
    # if not gpu:
    #     model.load_state_dict(torch.load(model_dir), map_location=lambda storage, loc: storage)
    #     # model = torch.load(model_dir, map_location=lambda storage, loc: storage)
    # else:
    model.load_state_dict(torch.load(model_dir))
        # model = torch.load(model_dir)
    #model = torch.load(model_dir)
    
    print("Decode %s data ..."%(name))
    start_time = time.time()
    speed, acc, p, r, f, pred_results, right_num = evaluate(data, model, name)
    end_time = time.time()
    time_cost = end_time - start_time
    if seg:
        print("%s: time:%.2fs, speed:%.2fst/s; acc: %.4f, p: %.4f, r: %.4f, f: %.4f"%(name, time_cost, speed, acc, p, r, f))
    else:
        print("%s: time:%.2fs, speed:%.2fst/s; acc: %.4f"%(name, time_cost, speed, acc))
    return pred_results

def load_model_decode_with_model(model, data, name, gpu, seg=True):
    data.HP_gpu = gpu
    
    print("Decode %s data ..."%(name))
    start_time = time.time()
    speed, acc, p, r, f, pred_results, right_num = evaluate(data, model, name)
    end_time = time.time()
    time_cost = end_time - start_time
    if seg:
        print("%s: time:%.2fs, speed:%.2fst/s; acc: %.4f, p: %.4f, r: %.4f, f: %.4f"%(name, time_cost, speed, acc, p, r, f))
    else:
        print("%s: time:%.2fs, speed:%.2fst/s; acc: %.4f"%(name, time_cost, speed, acc))
    return pred_results

In [ ]:
if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Tuning with bi-directional LSTM-CRF')
    parser.add_argument('--embedding',  help='Embedding for words', default='None')
    parser.add_argument('--status', choices=['train', 'test', 'decode'], help='update algorithm', default='train')
    #parser.add_argument('--savemodel', default="/content/drive/MyDrive/lattcie ner/fyz-lattcie/fyz_lattice_NER/model/2-222-saved_model2.lstmcrf")
    #parser.add_argument('--savedset', help='Dir of saved data setting', default="/content/drive/MyDrive/lattcie ner/fyz-lattcie/fyz_lattice_NER/ResumeNER/save-222.dset")
    parser.add_argument('--savemodel', default="/content/drive/MyDrive/lattcie ner/fyz-lattcie/cyx/final/lattice-lstm_model_final.model")
    parser.add_argument('--savedset', help='Dir of saved data setting', default="/content/drive/MyDrive/lattcie ner/fyz-lattcie/cyx/final/model-train-data_final.dset")
    #parser.add_argument('--train', default="/content/drive/MyDrive/lattcie ner/fyz-lattcie/fyz_lattice_NER/ResumeNER/train.char.bmes")
    #parser.add_argument('--dev', default="/content/drive/MyDrive/lattcie ner/fyz-lattcie/fyz_lattice_NER/ResumeNER/dev.char.bmes" )
    #parser.add_argument('--test', default="/content/drive/MyDrive/lattcie ner/fyz-lattcie/fyz_lattice_NER/ResumeNER/test.char.bmes")

    parser.add_argument('--train', default="/content/drive/MyDrive/lattcie ner/fyz-lattcie/cyx/final/train.txt")
    parser.add_argument('--dev', default="/content/drive/MyDrive/lattcie ner/fyz-lattcie/cyx/final/dev.txt" )
    parser.add_argument('--test', default="/content/drive/MyDrive/lattcie ner/fyz-lattcie/cyx/final/test.txt")

    #parser.add_argument('--train', default="test_data/fyz.train.embs")
    #parser.add_argument('--dev', default="test_data/fyz.dev.embs")
    #parser.add_argument('--test', default="test_data/fyz.test.embs")
    parser.add_argument('--seg', default="True") 
    parser.add_argument('--extendalphabet', default="True") 
    parser.add_argument('--raw') 
    #parser.add_argument('--loadmodel',default="/content/drive/MyDrive/lattcie ner/fyz-lattcie/fyz_lattice_NER/model/2-222-saved_model2.lstmcrf")
    parser.add_argument('--loadmodel',default="/content/drive/MyDrive/lattcie ner/fyz-lattcie/cyx/final/lattice-lstm_model_final.model")
    parser.add_argument('--output') 
    args = parser.parse_args()
   
    train_file = args.train
    dev_file = args.dev
    test_file = args.test
    raw_file = args.raw
    model_dir = args.loadmodel
    dset_dir = args.savedset
    output_file = args.output
    if args.seg.lower() == "true":
        seg = True 
    else:
        seg = False
    status = args.status.lower()

    save_model_dir = args.savemodel
    gpu = torch.cuda.is_available()

    char_emb = "/content/drive/MyDrive/lattcie ner/fyz-lattcie/fyz_lattice_NER/data/gigaword_chn.all.a2b.uni.ite50.vec"
    #char_emb = "/content/drive/MyDrive/lattcie ner/TCM_NER-master/TCM_NER-master/data/sgns.sikuquanshu.vec"
    bichar_emb = '/content/drive/MyDrive/lattcie ner/fyz-lattcie/fyz_lattice_NER/data/gigaword_chn.all.a2b.bi.ite50.vec'
    #bichar_emb = "/content/drive/MyDrive/lattcie ner/TCM_NER-master/TCM_NER-master/data/sgns.sikuquanshu.bigram.vec"
    gaz_file = "/content/drive/MyDrive/lattcie ner/fyz-lattcie/fyz_lattice_NER/data/ctb.50d.vec"
    # gaz_file = None
    # char_emb = None
    #bichar_emb = None

    print ("CuDNN:", torch.backends.cudnn.enabled)
    # gpu = False
    print ("GPU available:", gpu)
    print ("Status:", status)
    print ("Seg: ", seg)
    print ("Train file:", train_file)
    print ("Dev file:", dev_file)
    print ("Test file:", test_file)
    print ("Raw file:", raw_file)
    print ("Char emb:", char_emb)
    print ("Bichar emb:", bichar_emb)
    print ("Gaz file:",gaz_file)
    if status == 'train':
        print ("Model saved to:", save_model_dir)
    sys.stdout.flush()
    
    if status == 'train':
        data = Data()
        data.HP_gpu = gpu
        data.HP_use_char = False
        data.HP_batch_size = 10
        data.use_bigram = False
        data.gaz_dropout = 0.5
        data.norm_gaz_emb = False
        data.HP_fix_gaz_emb = False
        data_initialization(data, gaz_file, train_file, dev_file, test_file)

        data.generate_instance_with_gaz(train_file,'train')
        data.generate_instance_with_gaz(dev_file,'dev')
        data.generate_instance_with_gaz(test_file,'test')

        data.build_word_pretrain_emb(char_emb)
        data.build_biword_pretrain_emb(bichar_emb)
        data.build_gaz_pretrain_emb(gaz_file)
        #data = load_data_setting(dset_dir)
        model,train_log,dev_log = train(data, save_model_dir,dset_dir, seg)
    elif status == 'test':      
        data = load_data_setting(dset_dir)
        data.generate_instance_with_gaz(dev_file,'dev')
        load_model_decode(model_dir, data , 'dev', gpu, seg)
        data.generate_instance_with_gaz(test_file,'test')
        load_model_decode(model_dir, data, 'test', gpu, seg)
    elif status == 'decode':       
        data = load_data_setting(dset_dir)
        data.generate_instance_with_gaz(raw_file,'raw')
        decode_results = load_model_decode(model_dir, data, 'raw', gpu, seg)
        data.write_decoded_results(output_file, decode_results, 'raw')
    else:
        print ("Invalid argument! Please use valid arguments! (train/test/decode)")

torch.save(model,"/content/drive/MyDrive/lattcie ner/fyz-lattcie/cyx/final/321final.model")


'''filename='/content/drive/MyDrive/lattcie ner/fyz-lattcie/cyx/final/test_log.json'
with open(filename,'w',encoding='utf-8') as file_obj:
    json.dump(test_log,file_obj,ensure_ascii=False,indent = 4)'''

filename='/content/drive/MyDrive/lattcie ner/fyz-lattcie/cyx/final/train_log.json'
with open(filename,'w',encoding='utf-8') as file_obj:
    json.dump(train_log,file_obj,ensure_ascii=False,indent = 4)

filename='/content/drive/MyDrive/lattcie ner/fyz-lattcie/cyx/final/dev_log.json'
with open(filename,'w',encoding='utf-8') as file_obj:
    json.dump(dev_log,file_obj,ensure_ascii=False,indent = 4)

CuDNN: True
GPU available: True
Status: train
Seg:  True
Train file: /content/drive/MyDrive/lattcie ner/fyz-lattcie/cyx/final/train.txt
Dev file: /content/drive/MyDrive/lattcie ner/fyz-lattcie/cyx/final/dev.txt
Test file: /content/drive/MyDrive/lattcie ner/fyz-lattcie/cyx/final/test.txt
Raw file: None
Char emb: /content/drive/MyDrive/lattcie ner/fyz-lattcie/fyz_lattice_NER/data/gigaword_chn.all.a2b.uni.ite50.vec
Bichar emb: /content/drive/MyDrive/lattcie ner/TCM_NER-master/TCM_NER-master/data/gigaword_chn.all.a2b.bi.ite50.vec
Gaz file: /content/drive/MyDrive/lattcie ner/fyz-lattcie/fyz_lattice_NER/data/ctb.50d.vec
Model saved to: /content/drive/MyDrive/lattcie ner/fyz-lattcie/cyx/final/lattice-lstm_model_final.model
Load gaz file:  /content/drive/MyDrive/lattcie ner/fyz-lattcie/fyz_lattice_NER/data/ctb.50d.vec  total size: 704368
gaz alphabet size: 4235
gaz alphabet size: 4495
gaz alphabet size: 4632
build word pretrain emb...
Embedding:
     pretrain word:11327, prefect match:1696, ca

/content/drive/MyDrive/lattcie ner/fyz-lattcie/fyz_lattice_NER/model/latticelstm.py:105: UserWarning: nn.init.orthogonal is now deprecated in favor of nn.init.orthogonal_.
  init.orthogonal(self.weight_ih.data)
/content/drive/MyDrive/lattcie ner/fyz-lattcie/fyz_lattice_NER/model/latticelstm.py:106: UserWarning: nn.init.orthogonal is now deprecated in favor of nn.init.orthogonal_.
  init.orthogonal(self.alpha_weight_ih.data)
/content/drive/MyDrive/lattcie ner/fyz-lattcie/fyz_lattice_NER/model/latticelstm.py:118: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(self.bias.data, val=0)
/content/drive/MyDrive/lattcie ner/fyz-lattcie/fyz_lattice_NER/model/latticelstm.py:119: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(self.alpha_bias.data, val=0)
/content/drive/MyDrive/lattcie ner/fyz-lattcie/fyz_lattice_NER/model/latticelstm.py:37: UserWarning: nn.init.orthogonal is now deprecated in favor of nn

Instance: 500; Time: 13.88s; loss: 5101.4219; acc: 3642.0/6446.0=0.5650
Instance: 1000; Time: 16.35s; loss: 1787.6117; acc: 8997.0/12900.0=0.6974
Instance: 1500; Time: 14.03s; loss: 1417.7816; acc: 14776.0/19702.0=0.7500
Instance: 2000; Time: 13.38s; loss: 929.8085; acc: 20438.0/26119.0=0.7825
Instance: 2500; Time: 13.34s; loss: 885.6719; acc: 26135.0/32494.0=0.8043
Instance: 3000; Time: 13.31s; loss: 763.6597; acc: 31952.0/38973.0=0.8198
Instance: 3500; Time: 13.33s; loss: 782.4998; acc: 37734.0/45402.0=0.8311
Instance: 4000; Time: 13.61s; loss: 669.3196; acc: 43738.0/51980.0=0.8414
Instance: 4500; Time: 13.75s; loss: 694.6201; acc: 49745.0/58610.0=0.8487
Instance: 5000; Time: 13.54s; loss: 649.2617; acc: 55728.0/65163.0=0.8552
Instance: 5500; Time: 13.37s; loss: 664.3034; acc: 61648.0/71677.0=0.8601
Instance: 6000; Time: 13.84s; loss: 681.6650; acc: 67844.0/78459.0=0.8647
Instance: 6500; Time: 13.58s; loss: 621.3430; acc: 73924.0/85096.0=0.8687
Instance: 7000; Time: 13.20s; loss: 504

__main__:185: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
__main__:186: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
__main__:187: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
__main__:188: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
__main__:208: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/content/drive/MyDrive/lattcie ner/fyz-lattcie/fyz_lattice_NER/model/latticelstm.py:263: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  word_var = autograd.Variable(torch.LongTensor(skip_input_[t][0]),volatile =  volatile_flag)


gold_num =  11703  pred_num =  10830  right_num =  9079
Dev: time: 69.69s, speed: 88.96st/s; acc: 0.9545, p: 0.8383, r: 0.7758, f: 0.8058
Exceed previous best f score: -1
gold_num =  7806  pred_num =  7199  right_num =  5965
Test: time: 44.77s, speed: 92.43st/s; acc: 0.9523, p: 0.8286, r: 0.7642, f: 0.7951
Epoch: 1/25
 Learning rate is setted as: 0.014249999999999999
Instance: 500; Time: 13.12s; loss: 435.0030; acc: 6134.0/6487.0=0.9456
Instance: 1000; Time: 13.09s; loss: 410.2589; acc: 12190.0/12891.0=0.9456
Instance: 1500; Time: 13.09s; loss: 343.2539; acc: 18239.0/19237.0=0.9481
Instance: 2000; Time: 13.10s; loss: 396.2875; acc: 24248.0/25586.0=0.9477
Instance: 2500; Time: 13.55s; loss: 448.4299; acc: 30474.0/32229.0=0.9455
Instance: 3000; Time: 13.58s; loss: 407.7635; acc: 36664.0/38751.0=0.9461
Instance: 3500; Time: 13.39s; loss: 451.5262; acc: 42814.0/45288.0=0.9454
Instance: 4000; Time: 13.32s; loss: 403.1479; acc: 49023.0/51877.0=0.9450
Instance: 4500; Time: 13.46s; loss: 397.8

In [ ]:
if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Tuning with bi-directional LSTM-CRF')
    parser.add_argument('--embedding',  help='Embedding for words', default='None')
    parser.add_argument('--status', choices=['train', 'test', 'decode'], help='update algorithm', default='train')
    #parser.add_argument('--savemodel', default="/content/drive/MyDrive/lattcie ner/fyz-lattcie/fyz_lattice_NER/model/2-222-saved_model2.lstmcrf")
    #parser.add_argument('--savedset', help='Dir of saved data setting', default="/content/drive/MyDrive/lattcie ner/fyz-lattcie/fyz_lattice_NER/ResumeNER/save-222.dset")
    parser.add_argument('--savemodel', default="/content/drive/MyDrive/lattcie ner/fyz-lattcie/cyx/final/lattice-lstm_model_final.model")
    parser.add_argument('--savedset', help='Dir of saved data setting', default="/content/drive/MyDrive/lattcie ner/fyz-lattcie/cyx/final/model-train-data_final.dset")
    #parser.add_argument('--train', default="/content/drive/MyDrive/lattcie ner/fyz-lattcie/fyz_lattice_NER/ResumeNER/train.char.bmes")
    #parser.add_argument('--dev', default="/content/drive/MyDrive/lattcie ner/fyz-lattcie/fyz_lattice_NER/ResumeNER/dev.char.bmes" )
    #parser.add_argument('--test', default="/content/drive/MyDrive/lattcie ner/fyz-lattcie/fyz_lattice_NER/ResumeNER/test.char.bmes")

    parser.add_argument('--train', default="/content/drive/MyDrive/lattcie ner/fyz-lattcie/cyx/final/train.txt")
    parser.add_argument('--dev', default="/content/drive/MyDrive/lattcie ner/fyz-lattcie/cyx/final/dev.txt" )
    parser.add_argument('--test', default="/content/drive/MyDrive/lattcie ner/fyz-lattcie/cyx/final/test.txt")

    #parser.add_argument('--train', default="test_data/fyz.train.embs")
    #parser.add_argument('--dev', default="test_data/fyz.dev.embs")
    #parser.add_argument('--test', default="test_data/fyz.test.embs")
    parser.add_argument('--seg', default="True") 
    parser.add_argument('--extendalphabet', default="True") 
    parser.add_argument('--raw') 
    #parser.add_argument('--loadmodel',default="/content/drive/MyDrive/lattcie ner/fyz-lattcie/fyz_lattice_NER/model/2-222-saved_model2.lstmcrf")
    parser.add_argument('--loadmodel',default="/content/drive/MyDrive/lattcie ner/fyz-lattcie/cyx/final/lattice-lstm_model_final.model")
    parser.add_argument('--output') 
    args = parser.parse_args()
   
    train_file = args.train
    dev_file = args.dev
    test_file = args.test
    raw_file = args.raw
    model_dir = args.loadmodel
    dset_dir = args.savedset
    output_file = args.output
    if args.seg.lower() == "true":
        seg = True 
    else:
        seg = False
    status = args.status.lower()

    save_model_dir = args.savemodel
    gpu = torch.cuda.is_available()

    char_emb = "/content/drive/MyDrive/lattcie ner/fyz-lattcie/fyz_lattice_NER/data/gigaword_chn.all.a2b.uni.ite50.vec"
    #char_emb = "/content/drive/MyDrive/lattcie ner/TCM_NER-master/TCM_NER-master/data/sgns.sikuquanshu.vec"
    bichar_emb = '/content/drive/MyDrive/lattcie ner/TCM_NER-master/TCM_NER-master/data/gigaword_chn.all.a2b.bi.ite50.vec'
    #bichar_emb = "/content/drive/MyDrive/lattcie ner/TCM_NER-master/TCM_NER-master/data/sgns.sikuquanshu.bigram.vec"
    gaz_file = "/content/drive/MyDrive/lattcie ner/fyz-lattcie/fyz_lattice_NER/data/ctb.50d.vec"
    # gaz_file = None
    # char_emb = None
    #bichar_emb = None

    print ("CuDNN:", torch.backends.cudnn.enabled)
    # gpu = False
    print ("GPU available:", gpu)
    print ("Status:", status)
    print ("Seg: ", seg)
    print ("Train file:", train_file)
    print ("Dev file:", dev_file)
    print ("Test file:", test_file)
    print ("Raw file:", raw_file)
    print ("Char emb:", char_emb)
    print ("Bichar emb:", bichar_emb)
    print ("Gaz file:",gaz_file)
    if status == 'train':
        print ("Model saved to:", save_model_dir)
    sys.stdout.flush()
    
    if status == 'train':
        data = Data()
        data.HP_gpu = gpu
        data.HP_use_char = True
        data.HP_batch_size = 10
        data.use_bigram = True
        data.gaz_dropout = 0.5
        data.norm_gaz_emb = False
        data.HP_fix_gaz_emb = False
        data_initialization(data, gaz_file, train_file, dev_file, test_file)

        data.generate_instance_with_gaz(train_file,'train')
        data.generate_instance_with_gaz(dev_file,'dev')
        data.generate_instance_with_gaz(test_file,'test')

        data.build_word_pretrain_emb(char_emb)
        data.build_biword_pretrain_emb(bichar_emb)
        data.build_gaz_pretrain_emb(gaz_file)
        #data = load_data_setting(dset_dir)
        model,train_log,dev_log = train(data, save_model_dir,dset_dir, seg)
    elif status == 'test':      
        data = load_data_setting(dset_dir)
        data.generate_instance_with_gaz(dev_file,'dev')
        load_model_decode(model_dir, data , 'dev', gpu, seg)
        data.generate_instance_with_gaz(test_file,'test')
        load_model_decode(model_dir, data, 'test', gpu, seg)
    elif status == 'decode':       
        data = load_data_setting(dset_dir)
        data.generate_instance_with_gaz(raw_file,'raw')
        decode_results = load_model_decode(model_dir, data, 'raw', gpu, seg)
        data.write_decoded_results(output_file, decode_results, 'raw')
    else:
        print ("Invalid argument! Please use valid arguments! (train/test/decode)")

torch.save(model,"/content/drive/MyDrive/lattcie ner/fyz-lattcie/cyx/final/321final.model")


'''filename='/content/drive/MyDrive/lattcie ner/fyz-lattcie/cyx/final/test_log.json'
with open(filename,'w',encoding='utf-8') as file_obj:
    json.dump(test_log,file_obj,ensure_ascii=False,indent = 4)'''

filename='/content/drive/MyDrive/lattcie ner/fyz-lattcie/cyx/final/train_log.json'
with open(filename,'w',encoding='utf-8') as file_obj:
    json.dump(train_log,file_obj,ensure_ascii=False,indent = 4)

filename='/content/drive/MyDrive/lattcie ner/fyz-lattcie/cyx/final/dev_log.json'
with open(filename,'w',encoding='utf-8') as file_obj:
    json.dump(dev_log,file_obj,ensure_ascii=False,indent = 4)

CuDNN: True
GPU available: True
Status: train
Seg:  True
Train file: /content/drive/MyDrive/lattcie ner/fyz-lattcie/cyx/final/train.txt
Dev file: /content/drive/MyDrive/lattcie ner/fyz-lattcie/cyx/final/dev.txt
Test file: /content/drive/MyDrive/lattcie ner/fyz-lattcie/cyx/final/test.txt
Raw file: None
Char emb: /content/drive/MyDrive/lattcie ner/fyz-lattcie/fyz_lattice_NER/data/gigaword_chn.all.a2b.uni.ite50.vec
Bichar emb: /content/drive/MyDrive/lattcie ner/TCM_NER-master/TCM_NER-master/data/gigaword_chn.all.a2b.bi.ite50.vec
Gaz file: /content/drive/MyDrive/lattcie ner/fyz-lattcie/fyz_lattice_NER/data/ctb.50d.vec
Model saved to: /content/drive/MyDrive/lattcie ner/fyz-lattcie/cyx/final/lattice-lstm_model_final.model
Load gaz file:  /content/drive/MyDrive/lattcie ner/fyz-lattcie/fyz_lattice_NER/data/ctb.50d.vec  total size: 704368
gaz alphabet size: 4255
gaz alphabet size: 4474
gaz alphabet size: 4632
build word pretrain emb...
Embedding:
     pretrain word:11327, prefect match:1696, ca

/content/drive/MyDrive/lattcie ner/fyz-lattcie/fyz_lattice_NER/model/latticelstm.py:105: UserWarning: nn.init.orthogonal is now deprecated in favor of nn.init.orthogonal_.
  init.orthogonal(self.weight_ih.data)
/content/drive/MyDrive/lattcie ner/fyz-lattcie/fyz_lattice_NER/model/latticelstm.py:106: UserWarning: nn.init.orthogonal is now deprecated in favor of nn.init.orthogonal_.
  init.orthogonal(self.alpha_weight_ih.data)
/content/drive/MyDrive/lattcie ner/fyz-lattcie/fyz_lattice_NER/model/latticelstm.py:118: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(self.bias.data, val=0)
/content/drive/MyDrive/lattcie ner/fyz-lattcie/fyz_lattice_NER/model/latticelstm.py:119: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(self.alpha_bias.data, val=0)
/content/drive/MyDrive/lattcie ner/fyz-lattcie/fyz_lattice_NER/model/latticelstm.py:37: UserWarning: nn.init.orthogonal is now deprecated in favor of nn

build LatticeLSTM...  backward , Fix emb: False  gaz drop: 0.5
load pretrain word emb... (4632, 50)
build batched crf...
finished built model.
Epoch: 0/25
 Learning rate is setted as: 0.015


/content/drive/MyDrive/lattcie ner/fyz-lattcie/fyz_lattice_NER/model/crf.py:94: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  ../aten/src/ATen/native/IndexingUtils.h:30.)
  masked_cur_partition = cur_partition.masked_select(mask_idx)
/content/drive/MyDrive/lattcie ner/fyz-lattcie/fyz_lattice_NER/model/crf.py:99: UserWarning: masked_scatter_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/cuda/IndexKernel.cu:447.)
  partition.masked_scatter_(mask_idx, masked_cur_partition)
/content/drive/MyDrive/lattcie ner/fyz-lattcie/fyz_lattice_NER/model/crf.py:243: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  ../aten/src/ATen/native/IndexingUtils.h:30.)
  tg_energy = tg_energy.masked_select(mask.transpose(1,0

Instance: 500; Time: 13.99s; loss: 5004.6923; acc: 3819.0/6556.0=0.5825
Instance: 1000; Time: 13.18s; loss: 1793.5605; acc: 9200.0/12971.0=0.7093
Instance: 1500; Time: 13.05s; loss: 1230.3523; acc: 14756.0/19401.0=0.7606
Instance: 2000; Time: 13.55s; loss: 1058.6588; acc: 20653.0/26115.0=0.7908
Instance: 2500; Time: 12.93s; loss: 752.6873; acc: 26385.0/32435.0=0.8135
Instance: 3000; Time: 13.53s; loss: 772.8881; acc: 32415.0/39071.0=0.8296
Instance: 3500; Time: 13.39s; loss: 768.8669; acc: 38373.0/45660.0=0.8404
Instance: 4000; Time: 13.22s; loss: 724.7758; acc: 44312.0/52218.0=0.8486
Instance: 4500; Time: 13.00s; loss: 644.7211; acc: 50133.0/58628.0=0.8551
Instance: 5000; Time: 13.46s; loss: 686.8496; acc: 56096.0/65238.0=0.8599
Instance: 5500; Time: 13.68s; loss: 615.2045; acc: 62236.0/71898.0=0.8656
Instance: 6000; Time: 13.55s; loss: 590.0018; acc: 68370.0/78579.0=0.8701
Instance: 6500; Time: 13.16s; loss: 639.2688; acc: 74299.0/85014.0=0.8740
Instance: 7000; Time: 12.99s; loss: 55

__main__:185: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
__main__:186: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
__main__:187: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
__main__:188: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
__main__:208: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/content/drive/MyDrive/lattcie ner/fyz-lattcie/fyz_lattice_NER/model/latticelstm.py:263: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  word_var = autograd.Variable(torch.LongTensor(skip_input_[t][0]),volatile =  volatile_flag)


gold_num =  11549  pred_num =  11515  right_num =  9474
Dev: time: 66.90s, speed: 92.67st/s; acc: 0.9563, p: 0.8228, r: 0.8203, f: 0.8215
Exceed previous best f score: -1
gold_num =  7933  pred_num =  7930  right_num =  6392
Test: time: 47.08s, speed: 90.37st/s; acc: 0.9522, p: 0.8061, r: 0.8057, f: 0.8059
Epoch: 1/25
 Learning rate is setted as: 0.014249999999999999
Instance: 500; Time: 13.70s; loss: 450.0065; acc: 6353.0/6767.0=0.9388
Instance: 1000; Time: 13.31s; loss: 410.7344; acc: 12560.0/13357.0=0.9403
Instance: 1500; Time: 12.69s; loss: 315.0593; acc: 18459.0/19528.0=0.9453
Instance: 2000; Time: 12.77s; loss: 306.2908; acc: 24498.0/25854.0=0.9476
Instance: 2500; Time: 13.45s; loss: 349.0530; acc: 30784.0/32430.0=0.9492
Instance: 3000; Time: 13.27s; loss: 374.0533; acc: 36996.0/38967.0=0.9494
Instance: 3500; Time: 13.97s; loss: 391.3718; acc: 43391.0/45699.0=0.9495
Instance: 4000; Time: 13.37s; loss: 412.0422; acc: 49627.0/52272.0=0.9494
Instance: 4500; Time: 13.55s; loss: 343.4

In [ ]:
speed, acc, p, r, f, _, right_num = evaluate(data, model, "test")

__main__:185: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
__main__:186: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
__main__:187: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
__main__:188: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
__main__:208: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/content/drive/MyDrive/lattcie ner/fyz-lattcie/fyz_lattice_NER/model/latticelstm.py:263: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  word_var = autograd.Variable(torch.LongTensor(skip_input_[t][0]),volatile =  volatile_flag)
/content/drive/MyDrive/lattcie ner/fyz-lattcie/fyz_lattice_NER/model/crf.py:156: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,plea

gold_num =  7933  pred_num =  7808  right_num =  7522


In [ ]:
filename='/content/drive/MyDrive/lattcie ner/fyz-lattcie/cyx/final/test_log.json'
with open(filename,'w',encoding='utf-8') as file_obj:
    json.dump(_,file_obj,ensure_ascii=False,indent = 4)

In [ ]:
torch.save(model,"/content/drive/MyDrive/lattcie ner/fyz-lattcie/cyx/final/final.model")

In [ ]:
f=open('/content/drive/MyDrive/lattcie ner/fyz-lattcie/cyx/final/log.txt','w',encoding='utf-8')
for i in dev_log:
  f.write(str(i)+'\n')
f.close()

In [ ]:
speed, acc, p, r, f, pred = evaluate(data, model, "test")

__main__:184: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
__main__:185: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
__main__:186: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
__main__:187: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
__main__:207: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/content/drive/MyDrive/lattcie ner/fyz-lattcie/fyz_lattice_NER/model/latticelstm.py:263: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  word_var = autograd.Variable(torch.LongTensor(skip_input_[t][0]),volatile =  volatile_flag)
/content/drive/MyDrive/lattcie ner/fyz-lattcie/fyz_lattice_NER/model/crf.py:156: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,plea

gold_num =  7551  pred_num =  7500  right_num =  6922
